<img src = "https://drive.google.com/uc?export=view&id=1X3dvBPrNx6LbvsOJsYoZDOiUUwpYl-2v" alt = "Encabezado MLDS" width = "100%">  </img>

#**Reducción de la dimensionalidad**
---

En muchas tareas de análisis del mundo real el número de características a considerar es muy grande, dificultando el análisis por medio de visualización u otras tareas como el preprocesamiento o el modelado. La selección de las características es uno de los procesos de toma de decisiones más importantes en el aprendizaje automático, y en su desarrollo se han considerado técnicas de preprocesamiento que permiten reducir el número de dimensiones conservando la mayor cantidad de información posible. Este *notebook* se centra en esa necesidad y presenta la técnica de análisis de componentes principales.

# **1. Dependencias**
---
Importamos las librerías necesarias y definimos algunas funciones básicas de visualización que vamos a usar en algunos ejemplos.


### **1.1. Dependencias**
---
Para la construcción de modelos y ejecución de procedimientos metodológicos de aprendizaje automático, utilizaremos la librería _Scikit-learn_ (**`sklearn`**) y varias de sus funciones y conjuntos de datos.

In [ ]:
# Actualizamos scikit-learn a la última versión
!pip install -U scikit-learn

# Importamos scikit-learn
import sklearn

Importamos además algunas librerías básicas y configuraciones de *Python*.

In [ ]:
# Librerías básicas NumPy, Pandas, Matplotlib y Seaborn.
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Librería de visualización interactiva - Plotly
!pip install -U plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Configuraciones para las librerías y módulos usados.

# Ignoramos las advertencias o warnings.
import warnings
warnings.simplefilter(action='ignore')

# Configuramos el formato por defecto de la
# librería de visualización Matplotlib.
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
mpl.rcParams['figure.dpi'] = 105
mpl.rcParams['figure.figsize'] = (9, 7)
sns.set_theme()

In [ ]:
# Versiones de las librerías usadas.

!python --version
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('Matplotlib', mpl.__version__)
print('Seaborn', sns.__version__)
print('Plotly', plotly.__version__)
print('Scikit-learn', sklearn.__version__)

Esta actividad se realizó con las siguientes versiones:
*  *Python*: 3.7.10
*  *NumPy*:  1.19.5
*  *Pandas*: 1.1.5
*  *Matplotlib*:  3.2.2
*  *Seaborn*:  0.11.1
*  *Plotly*: 4.14.3
*  *Scikit-learn*: 0.24.1

### **1.2. Funciones de utilidad y visualización**
---

Para ilustrar los ejemplos discutidos en este material utilizaremos algunas funciones que permiten visualizar de manera general los datos, junto a las funciones de predicción obtenidas con cada modelo.

> **Nota**: *Matplotlib*, *Seaborn* y *Plotly* se encuentran por fuera del alcance de este módulo. No es necesario que entienda estas funciones en detalle para sacar partido del resto del contenido puesto a su disposición. Usted decide si leer o no estas funciones en profundidad. Si decide omitir esta sección, continúe directamente con la siguiente sección, en donde se discutirán los conjuntos de datos que vamos a utilizar.

In [ ]:
# Función para visualizar un conjunto de datos en 2D

def plot_data(X, y):
    y_unique = np.unique(y)
    colors = plt.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        plt.scatter(this_X[:, 0], this_X[:, 1],  color=color,
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    plt.legend(loc="best")

In [ ]:
# Grafica de la proyección en 2 dimensiones obtenida con PCA.

def plot_pca(X, Xr, title = "Proyección vectorial", components = None):

  plt.figure(dpi = 110, figsize = (7,6))
  plt.title(title)
  plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="Datos originales")
  plt.scatter(Xr[:,0], Xr[:,1], color="red", alpha=.5, label="Datos reconstruidos")

  plt.axis('equal')

  for i in range(len(X)):
    plt.plot([X[i, 0], Xr[i, 0]], [X[i, 1], Xr[i, 1]], ls = '--', color = '#333333')

  x_lo, x_hi = plt.xlim()
  y_lo, y_hi = plt.ylim()

  if not components is None:
    for i in range(len(components)):
      v = components[i]
      plt.plot([-100* v[0], 0, 100* v[0]], [-100 * v[1], 0, 100 * v[1]], lw=2, ls = '--', label=f"Componente principal {i + 1}")

  plt.ylim([-1.5, 1.5])
  plt.xlim([-1.5, 1.5])
  plt.legend(loc="center left", bbox_to_anchor=(1.01,.5));

In [ ]:
# Gráfica de la varianza explicada acumulada.

def cumulative_explained_variance_plot(expl_variance):

  cum_var_exp = np.cumsum(expl_variance)

  plt.figure(dpi = 100, figsize = (8, 6))
  plt.title('Curva acumulativa de la varianza explicada VS n° de componentes principales',
            fontdict= dict(family ='serif', size = 16))
  plt.xlabel('Número de componentes principales',
             fontdict= dict(family ='serif', size = 14))
  plt.ylabel('Varianza explicada acumulativa',
             fontdict= dict(family ='serif', size = 14))

  nc = np.arange(1, expl_variance.shape[0] + 1)

  plt.plot(nc, cum_var_exp, '--r')
  plt.plot(nc, cum_var_exp, 'c*', ms = 5)
  plt.show()

In [ ]:
# Gráfica de las imágenes originales y reconstruidas.

def show_img_matrix_pca(X, Xr):
  plt.figure(figsize=(10,6), dpi = 105)
  for i in range(6):
    k = np.random.randint(len(X))

    plt.subplot(3,6,i+1)
    plt.imshow(Xr[k].reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

    plt.subplot(3,6,6+i+1)
    plt.imshow(X[k].reshape(28,28), cmap=plt.cm.Greys_r)
    plt.xticks([]); plt.yticks([])

# **2. Conjuntos de datos**
---

Para los ejemplos desarrollados en el transcurso de material, se usarán datos de  *Scikit-Learn* de carácter real (usando *Loaders* y otras fuentes) y sintético (usando *Generators*).

### **2.1. Conjunto de datos *Wine***
---

El conjunto de datos *Wine* es uno de los más populares en el área del aprendizaje automático. Consiste en datos de un análisis químico de vinos de 3 cultivos diferentes de una región de Italia, y es comúnmente usado en tareas de clasificación. Hace parte del repositorio de *Machine Learning* de la Universidad de California en Irvine (UCI) y en este material se cargará con el método **`load_wine`** de **`sklearn.datasets`**.

In [ ]:
# Conjuntos de datos - Wine
from sklearn.datasets import load_wine

wine = load_wine(return_X_y = False)

X_wine = wine.data
y_wine = wine.target

In [ ]:
# Características del dataset
wine.feature_names

In [ ]:
# Primeras 5 observaciones de las características de Wine
pd.DataFrame(X_wine, columns = wine.feature_names).head()

In [ ]:
# Primeras 5 observaciones de la variable objetivo.
y_wine[:5]

In [ ]:
# Conteo de cada categoría de la variable objetivo
pd.value_counts(y_wine)

### **2.2. Digitos escritos a mano MNIST**
---
El *dataset* **MNIST** es un conjunto de datos de $60000$ imágenes de dígitos escritos a mano de $28 \times 28$ píxeles. Es usada comúnmente en tareas de clasificación, donde la variable objetivo corresponde a un dígito de $0$ a $9$, y con $784$ características, que corresponden a cada píxel de la imagen. En esta ocasión, usaremos la función **`fetch_openml`** de **`sklearn.datasets`** para descargar el conjunto de datos del repositorio [OpenML.org](https://www.openml.org/d/554).

> **Nota:** la carga del conjunto de datos puede tardar cerca de un minuto.

In [ ]:
#Conjuntos de datos - Repositorio OpenML
from sklearn.datasets import fetch_openml

# Descargamos el dataset desde OpenML. Este proceso puede tardar un poco.
mnist = fetch_openml('mnist_784', as_frame = False)

In [ ]:
X_mnist = mnist.data
y_mnist = mnist.target

In [ ]:
# Primeras 5 observaciones.
pd.DataFrame(X_mnist, columns = mnist.feature_names).head()

Cada número corresponde a un valor entre $0$ y $255$ con la intensidad del color, representado generalmente con escala de grises:

In [ ]:
# Visualización de la primera imagen del dataset.
plt.axis('off')
plt.imshow(mnist.data[0].reshape(28,28), cmap = 'gray');

In [ ]:
# Primeras 5 clases de la variable objetivo.
y_mnist[:5]

In [ ]:
# Conteo de la distribución de digitos
pd.value_counts(y_mnist)

#**3. Análisis de componentes principales (PCA)**
---

Inicialmente, consideraremos el caso en el que deseamos reducir un conjunto de datos de dos dimensiones a uno con solo una. Una alternativa es realizar una [proyección vectorial](https://matthew-brett.github.io/teaching/vector_projection.html) de nuestros datos en otro vector. La única dimensión de la nueva representación corresponderá a la magnitud del punto proyectado en la línea generada por el vector sobre el que se proyecta.
Esta idea se puede ver mejor de forma gráfica. A continuación, creamos unos datos 2D de manera aleatoria.


In [ ]:
# Creación de los datos sintéticos de ejemplo.

# Semilla aleatoria.
np.random.seed(1)

# Datos sintéticos correlacionados.
X = np.dot(np.random.random(size=(2, 2)),
           np.random.normal(size=(2, 40))).T + 10

# Centramos los datos en 0,0
X = X - np.mean(X, axis=0)

plt.figure(dpi = 110)
plt.title('Datos sintéticos correlacionados', fontdict = dict(family = 'serif', size = 18))
plt.scatter(X[:,0], X[:,1]);

En álgebra lineal, la proyección $proj_\vec{v} \vec{x}$ de un vector $\vec{x}$ en otro vector $\vec{v}$ se puede obtener mediante operaciones vectoriales:

$$proj_\vec{v} \vec{x} = c \vec{v}$$

con $c$ igual a:

$$c = \frac{\vec{v}\cdot \vec{x}}{||\vec{v}||^2}$$


$c$ es un escalar que refleja el punto de la proyección de $\vec{x}$ sobre $\vec{v}$. Este valor se puede interpretar como una representación de 1 dimensión sobre el vector $\vec{v}$.

Vamos a inspeccionar gráficamente algunas posibles proyecciones:

In [ ]:
#@title
#@markdown **Animación:** Ejecute esta celda para ver un ejemplo de proyección de un conjunto de datos de 2 dimensiones a un vector de 1 dimensión, con un registro de la varianza del vector resultante. Oprima el botón **'Reproducir'** para iniciar la animación.
def unit_vector(angle):
  return np.array([np.cos(angle), np.sin(angle)])

n = 120
angles = np.linspace(0, np.pi, n)
projs = []

frames_data = []
for i in range(n):
    angle = angles[i] # Ángulo entre 0 y pi.
    v = unit_vector(angle) # Vector sobre el cual se realizará la proyección.
    c = X.dot(v.reshape(-1,1))/(np.linalg.norm(v)**2) # Se calcula c con operación vectoriales de NumPy.
    Xp = c * v # Valor en dos dimensiones con la posición de la proyección del vector.
    frames_data.append( (Xp, angle, v, c))


Xp0, angle0, v0, c0 = frames_data[0]

fig = go.Figure(
    data = [
        go.Scatter(x = [-10* v0[0], 10* v0[0]], y = [-10 * v0[1], 10 * v0[1]], name = 'Vector de proyección', marker = dict(color = 'black'),
                   text = angle0),
        go.Scatter(x = [ -1.25], y = [ 0.75], text = f"<b>Ángulo:</b> {angle0:.4f}", textfont = dict(size = 20), textposition = 'top right',
                   showlegend= False, mode = 'text'),
        go.Scatter(x = [ -1.25], y = [ 0.65], text = f"<b>Varianza de la proyección:</b> = {np.var(c0):.4f}", textposition = 'top right',
                   textfont = dict(size = 20), showlegend= False, mode = 'text'),
            *[go.Scatter(x = [X[i, 0], Xp0[i, 0]], y = [X[i, 1], Xp0[i, 1]], line = dict(dash = 'dash'), showlegend= False,
                         marker = dict(color = 'grey')) for i in range(len(X))],
        go.Scatter(x = X[:,0], y= X[:,1], mode = 'markers', name = 'Datos originales', marker = dict(color = 'blue')),
        go.Scatter(x = Xp0[:,0], y= Xp0[:,1], mode = 'markers', name = 'Datos reconstruidos', marker = dict(color = 'red'))
        ],

    layout= go.Layout(
        height = 700,
        width = 900,
        xaxis=dict(range=[-2, 2], autorange=False),
        yaxis=dict(range=[-1, 1], autorange=False),
        title= dict(text = "<b>Varianza de las posibles proyecciones vectoriales</b>", font = dict(size = 25), xanchor = 'left'),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Reproducir",
                          method="animate",
                          args=[None,  dict(frame = dict(duration = 50, redraw = False))]
                          )])]
    ),
    frames  = [go.Frame(
        data = [
        go.Scatter(x = [-10* v[0], 10* v[0]], y = [-10 * v[1], 10 * v[1]], name = 'Vector de proyección', marker = dict(color = 'black')),
        go.Scatter(x = [ -1.25], y = [ 0.75], text = f"<b>Ángulo:</b> {angle:.4f}", textfont = dict(size = 20),
                   textposition = 'top right',
                   showlegend= False, mode = 'text'),
        go.Scatter(x = [ -1.25], y = [ 0.65], text = f"<b>Varianza de la proyección:</b> = {np.var(c):.4f}", textposition = 'top right',
                   textfont = dict(size = 20), showlegend= False, mode = 'text'),
                *[go.Scatter(x = [X[i, 0], Xp[i, 0]], y = [X[i, 1], Xp[i, 1]], line = dict(dash = 'dash'), showlegend= False,
                             marker = dict(color = 'grey')) for i in range(len(X))],
        go.Scatter(x = X[:,0], y= X[:,1], mode = 'markers', name = 'Datos originales', marker = dict(color = 'blue')),
        go.Scatter(x = Xp[:,0], y= Xp[:,1], mode = 'markers', name = 'Datos reconstruidos', marker = dict(color = 'red'))
                ]) for Xp, angle, v, c in frames_data]
)

fig.show()


Ahora note que las proyecciones que permiten distinguir más entre observaciones de los datos originales y que más se ajustan a ellos son las que tienen mayor **variabilidad o varianza**. Por lo general, queremos encontrar una proyección con menos dimensiones que preserve la máxima cantidad de variabilidad. Este es la idea detrás de la técnica **PCA**.

## **3.1. Análisis de componentes principales**
---
El **análisis de componentes principales** (llamado comúnmente por sus siglas en inglés [PCA](https://es.wikipedia.org/wiki/An%C3%A1lisis_de_componentes_principales)) es una técnica utilizada para describir un conjunto de datos en términos de nuevas variables (componentes) no correlacionadas. Las componentes son ordenadas por la cantidad de varianza original que son capaces de describir y es útil para reducir la dimensionalidad de un conjunto de datos.

La técnica *PCA* busca la proyección según la cual los datos queden mejor representados en términos de mínimos cuadrados, y convierte un conjunto de observaciones de variables posiblemente correlacionadas en un conjunto de valores de variables sin correlación lineal llamadas componentes principales. Esta técnica se emplea principalmente en el análisis exploratorio de datos y en la construcción de modelos predictivos.

**Limitaciones:**
  * Se asume que los datos observados son combinación lineal de una cierta base.
  * *PCA* utiliza los vectores propios de la matriz de covarianzas y sólo encuentra las direcciones de ejes en el espacio de variables considerando que los datos tienen una distribución normal.

<center><img src = "https://blog-c7ff.kxcdn.com/blog/wp-content/uploads/2018/07/pca.gif" alt = "PCA" width = "70%">  </img></center>


*Scikit-Learn* permite aplicar un algoritmo de *PCA* sobre un conjunto de datos mediante la función **`sklearn.decomposition.PCA`**. Esta recibe el argumento **`n_components`** que define la dimensión de la nueva representación.

Al igual que con otros estimadores y modelos de **`sklearn`**, este objeto cuenta con la función **`.fit()`** para ingresar los datos que se desean descomponer con PCA.

In [ ]:
# Análisis de componentes principales (PCA)
from sklearn.decomposition import PCA

pca = PCA(n_components=1)
pca.fit(X)

Para nuestro ejemplo, podemos usar el componente mayor para reducir la dimensionalidad de nuestros datos de $2$ dimensiones a solamente $1$.

Observe que:

$$\mathbf{X_t} = \mathbf{X} \cdot \mathbf{V_c}$$

donde:
- $\mathbf{X}$ son nuestros datos
- $\mathbf{V_c}$ es el vector de componentes seleccionados
- $\mathbf{X_t}$ son los datos transformados

En este caso nos estamos restringiendo a **transformaciones lineales**, es decir, rotaciones y escalado. Para realizar este proceso con *Scikit-Learn* utilizamos el método **`.transform()`**.


In [ ]:
# Transformación de los datos originales en la proyección de menor dimensión.
Xt = pca.transform(X)

# Datos con una dimensión (característica) menos.
print(X.shape, Xt.shape)

Es muy común que los datos usados en la obtención de los componentes (usando del método **`fit`**) sean los mismos que queramos transformar a una dimensión menor. Este proceso se puede realizar en un solo paso utilizando el método **`fit_transform`** de **`PCA`**.

In [ ]:
pca = PCA(n_components= 1)
Xft = pca.fit_transform(X)

# Datos con una dimensión menos. Igual que usar fit y después transform.
Xft.shape

In [ ]:
plt.figure(dpi = 105, figsize = (10,4))

# Datos originales

plt.subplot(1,2,1)
plt.scatter(X[:,0], X[:,1], color="blue", alpha=.5, label="$\mathbf{X}$: Datos originales");
plt.axis("equal"); plt.legend();

# Datos reducidos
plt.subplot(1,2,2)
plt.scatter(Xft, np.zeros(len(Xft)), color="red", alpha=.5, label="$\mathbf{X_t}$: Datos reducidos");
plt.axis("equal"); plt.legend();

También podemos reconstruir los datos a la dimensión original a partir de datos obtenidos mediante una transformación.

In [ ]:
Xr = pca.inverse_transform(Xt)

plot_pca(X, Xr,
         title = "Reconstrucción a partir del componente mayor",
         components = pca.components_)

Con el atributo **`components_`** podemos acceder a los vectores de los componentes principales obtenidos con PCA. Obtengamos $2$ componentes principales y veamos su representación gráfica.

In [ ]:
pca = PCA(n_components= 2)
Xp = pca.fit_transform(X)
Xr = pca.inverse_transform(Xp)

pca.components_

In [ ]:
# Representamos con vectores los componentes principales.
plot_pca(X,
         Xr,
         title = "Componentes principales",
         components = pca.components_)

##**3.2. Varianza explicada para las componentes principales**
---

En *PCA*, la varianza total es la suma de las varianzas de todos los componentes principales individuales.
Entonces, la **proporción de varianza explicada** por un componente principal es la relación entre la varianza de ese componente principal y la varianza total.
Por otro lado, si se quiere conocer el porcentaje de varianza total de varios componentes principales se puede obtener con la suma de sus varianzas y entre la varianza total.

Esto se hace generalmente con los componentes ordenados por mayor varianza explicada y se representa gráficamente con una gráfica de líneas con la **varianza explicada acumulada**.

In [ ]:
pca = PCA(n_components = 2)
Xt = pca.fit_transform(X)

La varianza explicada se obtiene del atributo  **`explained_variance_ratio_`**:

In [ ]:
pca.explained_variance_ratio_

El resultado indica que el primer componente explica el $97.6\%$ de la varianza y el segundo componente el $2.4\%$ restante.

En este caso, realizar la gráfica de la varianza acumulada no tiene mucho sentido pues solo contamos con dos columnas. Veamos la aplicación de PCA en un contexto real. Primero, cargamos el conjunto de datos ***Wine***:

In [ ]:
wine = load_wine()

labels = wine.target
classes = wine.target_names

wine_df = pd.DataFrame(wine.data, columns = wine.feature_names)
wine_df.head()

Ahora, realizamos un escalado de los datos, con media en 0 y varianza en 1. Esto se puede realizar con el método **`sklearn.preprocessing.StandardScaler`**.

In [ ]:
# Preprocesamiento (Reescalado estándar)
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_x.fit(wine_df.values)
X_scaled = sc_x.transform(wine_df.values)

In [ ]:
X_scaled.shape

Analizando la varianza explicada de las componentes principales obtenemos lo siguiente:

In [ ]:
# Si no se indica el número de componentes se usa la cantidad de columnas.
pca = PCA()
transf = pca.fit_transform(X_scaled)

varianza_expl = pca.explained_variance_ratio_

print(varianza_expl)

A continuación, graficamos la curva acumulativa de la varianza explicada versus el número de componentes principales.

In [ ]:
cumulative_explained_variance_plot(varianza_expl)

Podemos observar lo siguiente:
  * Las componentes principales $1$ y $2$ representan el $55,4\%$ de la varianza de los datos
  * Las $7$ primeras componentes principales representan el $89,3\%$ de la varianza de los datos

Podemos usar los componentes principales $1$ y $2$ para visualizar el conjunto de datos en $2$ dimensiones. Para hacer esto usamos el comando **`sklearn_pca.fit_transform`**:

In [ ]:
pca = PCA(n_components=2)
X_transf = pca.fit_transform(X_scaled)

plt.figure(figsize = (10, 8), dpi = 105)
plt.xlabel('Componente principal 1')
plt.ylabel('Componente principal 2')
plt.title('Vectores singulares más significativos después de la transformación lineal a través de PCA')

plot_data(X_transf, labels)

A pesar de ser un *dataset* de $13$ características, somos capaces de distinguir sus valores de forma gráfica en $2$ dimensiones usando sus componentes principales. Al ser los dos componentes más importantes, su visualización es la que contiene mayor varianza y es más fácil de distinguir entre sus observaciones. Ahora, consideremos una gráfica con los componentes principales $2$ y $3$.

In [ ]:
pca = PCA(n_components=3)
X_transf = pca.fit_transform(X_scaled)

plt.figure(figsize = (10, 8), dpi = 105)
plt.xlabel('Componente principal 2')
plt.ylabel('Componente principal 3')
plt.title('Vectores singulares más significativos después de la transformación lineal a través de PCA')
plot_data(X_transf[:,1:3],labels)

# **4. Aplicación en clasificación**
---
Ahora, veremos la aplicación de este método de reducción de la dimensionalidad en tareas de modelado como clasificación y agrupamiento, usando como datos de entrenamiento la representación obtenida con PCA.
> **¿Qué tan conveniente es este acercamiento al problema?**

En esta parte trabajaremos con una muestra del conjunto de imágenes de **MNIST** que contiene únicamente $1500$ imágenes de dígitos escritos a mano. Aplicaremos reducción de dimensionalidad y usaremos los datos reducidos con esta herramienta en tareas de clasificación.

In [ ]:
X_mnist = mnist.data[:1500]
y_mnist = mnist.target[:1500]
print("Dimensión de las imágenes y las clases: ", X_mnist.shape, y_mnist.shape)

Visualicemos una muestra aleatoria de imágenes del *dataset* con el método de visualización de imágenes de *Matplotlib* **`imshow`**.

In [ ]:
perm = np.random.permutation(range(X_mnist.shape[0]))[0:50]

random_imgs   = X_mnist[perm]
random_labels = y_mnist[perm]

fig = plt.figure(figsize=(10,6))
for i in range(random_imgs.shape[0]):
    ax=fig.add_subplot(5,10,i+1)
    plt.imshow(random_imgs[i].reshape(28,28), interpolation="nearest", cmap = plt.cm.Greys_r)
    ax.set_title(int(random_labels[i]))
    plt.xticks([]); plt.yticks([])

A continuación, graficamos la curva acumulativa de la varianza explicada versus el número de componentes principales:

In [ ]:
# PCA en MNIST
pca = PCA(n_components=None)
X_mnist_transf = pca.fit_transform(X_mnist)
varianza_expl = pca.explained_variance_ratio_

cum_var_exp = np.cumsum(varianza_expl)

cumulative_explained_variance_plot(varianza_expl)

In [ ]:
print(f'Primeras 10 componentes: {cum_var_exp[9]}')
print(f'Primeras 60 componentes: {cum_var_exp[59]}')
print(f'Primeras 100 componentes: {cum_var_exp[99]}')

Podemos observar lo siguiente:
  * Las $10$ primeras componentes principales representan el $49,2\%$ de la varianza de los datos
  * Las $60$ primeras componentes principales representan el $86,2\%$ de la varianza de los datos
  * Las $100$ primeras componentes principales representan el $92,3\%$ de la varianza de los datos

A continuación, usamos las primeras $10$ componentes principales y graficaremos la transformación inversa:

In [ ]:
pca = PCA(n_components=10)
Xp = pca.fit_transform(X_mnist)
Xr = pca.inverse_transform(Xp)

show_img_matrix_pca(X_mnist, Xr)

Los datos reconstruidos se asemejan bastante a los datos originales. Un proceso similar es el realizado en la [compresión de imágenes](https://en.wikipedia.org/wiki/Image_compression), en el cual se conserva la información más relevante y  necesaria para que sea reconocible.
Ahora usemos las primeras $60$ componentes principales:

In [ ]:
pca = PCA(n_components=60)
Xp = pca.fit_transform(X_mnist)
Xr = pca.inverse_transform(Xp)

show_img_matrix_pca(X_mnist, Xr)

Y con las primeras $100$ componentes principales:

In [ ]:
pca = PCA(n_components=100)
Xp = pca.fit_transform(X_mnist)
Xr = pca.inverse_transform(Xp)

show_img_matrix_pca(X_mnist, Xr)


Los datos resultantes no están organizados como en los pixeles de una imágen, sino que corresponden a la codificación numérica de la información más importante con menos dimensiones.

In [ ]:
print(Xr[0,:5])

Finalmente, realicemos el proceso de modelado de clasificación de las imágenes, y comparemos los resultados entre las dos opciones. En este caso usaremos un clasificador bayesiano con el método **`sklearn.naive_bayes.GaussianNB`**.

Los clasificadores *Naive Bayes* están basados en métodos de clasificación bayesiana y en el teorema de *Bayes*, que describe la relación de las probabilidades condicionales de cantidades estadísticas. Este método funciona de forma similar a los demás clasificadores vistos en el curso.

Veamos la efectividad de este modelo tanto para los datos originales como para la nueva representación obtenida con _PCA_.

In [ ]:
# Selección de modelos y validación cruzada.
from sklearn.model_selection import cross_val_score

# Métodos de modelado - Clasificación con NaiveBayes Gaussiano.
from sklearn.naive_bayes import GaussianNB

print(f"Accuracy con datos originales: {np.mean(cross_val_score(GaussianNB(), X_mnist, y_mnist, cv=5)):.4f}")
print(f"Accuracy con la nueva representación: {np.mean(cross_val_score(GaussianNB(), Xp, y_mnist, cv=5)):.4f}")

Podemos notar que al usar los datos de entrenamiento reducidos obtenemos una mayor precisión o *accuracy* promedio que al usar todos los datos originales.

# **Recursos adicionales**
---
Los siguientes enlaces corresponden a sitios en donde encontrará información muy útil para profundizar en el conocimiento de las funcionalidades de la librería *Scikit-learn* en la aplicación de técnicas de reducción de la dimensionalidad y análisis de componentes principales, además de material de apoyo teórico para reforzar estos conceptos:

* **Introductorios**
  * [A One-Stop Shop for Principal Component Analysis - Matt Brems](https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c)
  * [PCA example with Iris Data-set](https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html#sphx-glr-auto-examples-decomposition-plot-pca-iris-py)
  * [Manifold Hypothesis - Deep AI glossary](https://deepai.org/machine-learning-glossary-and-terms/manifold-hypothesis)
  * [Dimensionality Reduction, PCA Intro - AI Pool](https://ai-pool.com/a/s/dimensionality-reduction--pca-intro)
  * [Principal component analysis explained simply - BioTuring's Blog](https://blog.bioturing.com/2018/06/14/principal-component-analysis-explained-simply/)

* **Avanzados**
  * [Mathematics For Machine Learning Chapter 10 - Dimensionality Reduction with Principal Component Analysis](https://mml-book.github.io/book/mml-book.pdf)

  * [John Shlens - A Tutorial on Principal Component Analysis](https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf)


# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes:**
  * Miguel Angel Ortiz Marín
  * Alberto Nicolai Romero Martínez

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*